In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper Parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [3]:
# MNIST Dataset
train_dataset = dsets.MNIST(
    root='../../data/', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = dsets.MNIST(
    root='../../data/', train=False, transform=transforms.ToTensor())

In [4]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial states
        h0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())
        c0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size).cuda())

        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

In [6]:
rnn = RNN(input_size, hidden_size, num_layers, num_classes)
rnn.cuda()

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [7]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [8]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size)).cuda()
        labels = Variable(labels).cuda()

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' %
                  (epoch + 1, num_epochs, i + 1,
                   len(train_dataset) // batch_size, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5494
Epoch [1/2], Step [200/600], Loss: 0.2035
Epoch [1/2], Step [300/600], Loss: 0.1505
Epoch [1/2], Step [400/600], Loss: 0.3548
Epoch [1/2], Step [500/600], Loss: 0.1892
Epoch [1/2], Step [600/600], Loss: 0.1495
Epoch [2/2], Step [100/600], Loss: 0.0616
Epoch [2/2], Step [200/600], Loss: 0.1158
Epoch [2/2], Step [300/600], Loss: 0.1124
Epoch [2/2], Step [400/600], Loss: 0.1369
Epoch [2/2], Step [500/600], Loss: 0.1065
Epoch [2/2], Step [600/600], Loss: 0.1572


In [9]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size)).cuda()
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' %
      (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97 %


In [10]:
# Save the Model

In [11]:
torch.save(rnn.state_dict(), 'rnn.pkl')